In [13]:
import time
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic =True

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64

num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10

train_dataset = datasets.MNIST(root='data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = datasets.MNIST(root='data',train=False,transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

for images, labels in train_loader:
  print('Image batch dimensions:',images.shape)
  print('Image label dimensions:',labels.shape)
  break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [15]:
class MultilayerPerceptron(torch.nn.Module):
  def __init__(self,num_features,num_classes):
    super(MultilayerPerceptron,self).__init__()

    ###1st hidden layer
    self.linear_1 = torch.nn.Linear(num_features,num_hidden_1)
    self.linear_1.weight.detach().normal_(0.0,0.1)
    self.linear_1.bias.detach().zero_()
    ###2nd hidden layer
    self.linear_2 = torch.nn.Linear(num_hidden_1,num_hidden_2)
    self.linear_2.weight.detach().normal_(0.0,0.1)
    self.linear_2.bias.detach().zero_()
    ###output layer
    self.linear_out = torch.nn.Linear(num_hidden_2,num_classes)
    self.linear_out.weight.detach().normal_(0.0,0.1)
    self.linear_out.bias.detach().zero_()

  def forward(self,x):
    out = self.linear_1(x)
    out = F.relu(out)
    out = self.linear_2(out)
    out = F.relu(out)
    logits = self.linear_out(out)
    probas = F.log_softmax(logits, dim=1)
    return logits, probas
torch.manual_seed(random_seed)
model=MultilayerPerceptron(num_features=num_features,num_classes=num_classes)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [18]:
def compute_accuracy(net,data_loader):
  net.eval()
  correct_pred,num_examples = 0,0
  with torch.no_grad():
    for features, targets in data_loader:
      features = features.view(-1,28*28).to(device)
      targets = targets.to(device)
      logits, probas = net(features)
      _,predicted_labels = torch.max(probas,1)
      num_examples += targets.size(0)
      correct_pred +=(predicted_labels == targets).sum()
    return correct_pred.float()/num_examples*100
start_time = time.time()
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (features, targets) in enumerate(train_loader):
    features = features.view(-1, 28*28).to(device)
    targets = targets.to(device)
    logits, probas = model(features)
    cost = F.cross_entropy(logits,targets)
    optimizer.zero_grad()

    cost.backward()

    optimizer.step()
    if not batch_idx %50:
      print("epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f"%(epoch+1,num_epochs,batch_idx,len(train_loader),cost))
  with torch.set_grad_enabled(False):
    print("Epoch:%03d/%03d training accuracy: %.2f%%"%(epoch+1,num_epochs,compute_accuracy(model,train_loader)))
  print("Time elapsed: %.2f min"%((time.time()-start_time)/60))
print("Total Training Time: %.2f min" % ((time.time()-start_time)/60))

epoch: 001/010 | Batch 000/938 | Cost: 0.0911
epoch: 001/010 | Batch 050/938 | Cost: 0.2216
epoch: 001/010 | Batch 100/938 | Cost: 0.1300
epoch: 001/010 | Batch 150/938 | Cost: 0.1624
epoch: 001/010 | Batch 200/938 | Cost: 0.0288
epoch: 001/010 | Batch 250/938 | Cost: 0.1230
epoch: 001/010 | Batch 300/938 | Cost: 0.2051
epoch: 001/010 | Batch 350/938 | Cost: 0.2053
epoch: 001/010 | Batch 400/938 | Cost: 0.0668
epoch: 001/010 | Batch 450/938 | Cost: 0.0770
epoch: 001/010 | Batch 500/938 | Cost: 0.0875
epoch: 001/010 | Batch 550/938 | Cost: 0.1006
epoch: 001/010 | Batch 600/938 | Cost: 0.0962
epoch: 001/010 | Batch 650/938 | Cost: 0.0409
epoch: 001/010 | Batch 700/938 | Cost: 0.0295
epoch: 001/010 | Batch 750/938 | Cost: 0.0504
epoch: 001/010 | Batch 800/938 | Cost: 0.0809
epoch: 001/010 | Batch 850/938 | Cost: 0.1011
epoch: 001/010 | Batch 900/938 | Cost: 0.0670
Epoch:001/010 training accuracy: 96.91%
Time elapsed: 0.27 min
epoch: 002/010 | Batch 000/938 | Cost: 0.0676
epoch: 002/010 | 

In [19]:
print("Test accuracy: %.2f%%"%(compute_accuracy(model,test_loader)))

Test accuracy: 97.84%
